# Pre-Season Testing Analysis (2020-2025)

Extract car characteristics from pre-season testing to establish baseline performance.

**Goal:** Predict 2026 season performance using testing data only.

**Key Metrics:**
- Straightline speed (DRS zones, top speed)
- Corner performance (slow/medium/high speed)
- Tire degradation patterns
- Lap count & reliability

**Hypothesis:** Testing correlates with season performance, especially for new regs (2022, 2026).

In [1]:
import warnings
from pathlib import Path

import fastf1
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

warnings.filterwarnings('ignore')

import logging

logging.getLogger("fastf1").setLevel(logging.ERROR)

# Enable FastF1 cache
cache_dir = Path("../data/raw/.fastf1_cache")
cache_dir.mkdir(parents=True, exist_ok=True)
fastf1.Cache.enable_cache(str(cache_dir))

print("✓ Setup complete")

✓ Setup complete


## 1. Load Pre-Season Testing Data

FastF1 has testing data for Bahrain pre-season tests (usually 3 days).

In [2]:
def load_testing_sessions(year):
    """Load all pre-season testing sessions for a year."""
    print(f"Loading {year} pre-season testing...")
    
    try:
        schedule = fastf1.get_event_schedule(year)
        testing_events = schedule[schedule['EventFormat'] == 'testing']
        
        if len(testing_events) == 0:
            print(f"  ⚠️  No testing events found for {year}")
            return []
        
        sessions = []
        for idx, event in testing_events.iterrows():
            event_name = event['EventName']
            print(f"  Loading: {event_name}")
            
            # Testing usually has 3 sessions (Day 1, 2, 3)
            for session_num in [1, 2, 3]:
                try:
                    session = fastf1.get_session(year, event_name, session_num)
                    session.load(laps=True, telemetry=True)
                    sessions.append({
                        'year': year,
                        'event': event_name,
                        'day': session_num,
                        'session': session
                    })
                    print(f"    ✓ Day {session_num}: {len(session.laps)} laps")
                except Exception as e:
                    print(f"    ✗ Day {session_num}: {e}")
        
        return sessions
    
    except Exception as e:
        print(f"  ✗ Error loading {year}: {e}")
        return []

# Load testing data for 2020-2025
years = [2020, 2021, 2022, 2023, 2024, 2025]
all_testing_data = {}

for year in years:
    sessions = load_testing_sessions(year)
    if sessions:
        all_testing_data[year] = sessions

print(f"\n✓ Loaded testing data for {len(all_testing_data)} years")

Loading 2020 pre-season testing...
  Loading: Pre-Season Test 1
    ✓ Day 1: 584 laps
    ✓ Day 2: 868 laps
    ✓ Day 3: 372 laps
  Loading: Pre-Season Test 2
    ✓ Day 1: 584 laps
    ✓ Day 2: 868 laps
    ✓ Day 3: 372 laps
Loading 2021 pre-season testing...
  Loading: Pre-Season Test
    ✓ Day 1: 362 laps
    ✓ Day 2: 471 laps
    ✓ Day 3: 283 laps
Loading 2022 pre-season testing...
  Loading: Pre-Season Track Session
    ✓ Day 1: 80 laps
    ✓ Day 2: 471 laps
    ✓ Day 3: 422 laps
  Loading: Pre-Season Test
    ✓ Day 1: 413 laps
    ✓ Day 2: 554 laps
    ✓ Day 3: 360 laps
Loading 2023 pre-season testing...
  Loading: Pre-Season Testing
    ✓ Day 1: 469 laps
    ✓ Day 2: 512 laps
    ✓ Day 3: 341 laps
Loading 2024 pre-season testing...
  Loading: Pre-Season Testing
    ✓ Day 1: 488 laps
    ✓ Day 2: 521 laps
    ✓ Day 3: 355 laps
Loading 2025 pre-season testing...
  Loading: Pre-Season Testing
    ✓ Day 1: 472 laps
    ✓ Day 2: 353 laps
    ✓ Day 3: 440 laps

✓ Loaded testing data fo

## 2. Extract Car Characteristics

Use telemetry to extract performance metrics for each team.

In [3]:
def extract_testing_characteristics(session_data):
    """Extract car characteristics from testing session."""
    session = session_data['session']
    laps = session.laps
    
    # Filter to valid laps only
    clean_laps = laps[
        (laps['IsAccurate'] == True) & 
        (laps['TrackStatus'] == '1')  # Green flag
    ].copy()
    
    team_characteristics = {}
    
    for team in clean_laps['Team'].unique():
        team_laps = clean_laps[clean_laps['Team'] == team]
        
        if len(team_laps) < 5:  # Need minimum laps
            continue
        
        # Extract features from fastest lap
        try:
            fastest_lap = team_laps.pick_fastest()
            telemetry = fastest_lap.get_telemetry()
            
            if telemetry is None or len(telemetry) == 0:
                continue
            
            # Straightline speed (top speed at full throttle)
            full_throttle = telemetry[telemetry['Throttle'] == 100]
            max_speed = telemetry['Speed'].max()
            avg_full_throttle_speed = full_throttle['Speed'].mean() if len(full_throttle) > 0 else np.nan
            
            # Corner speeds (slow < 100, medium 100-200, high 200-250 km/h)
            corners = telemetry[telemetry['Speed'] < 250]
            slow_corners = corners[(corners['Speed'] >= 0) & (corners['Speed'] < 100)]
            medium_corners = corners[(corners['Speed'] >= 100) & (corners['Speed'] < 200)]
            high_corners = corners[(corners['Speed'] >= 200) & (corners['Speed'] < 250)]
            
            # Tire degradation proxy: lap time variance over stint
            lap_times = team_laps['LapTime'].dt.total_seconds()
            lap_time_std = lap_times.std()
            
            team_characteristics[team] = {
                'total_laps': len(team_laps),
                'fastest_lap': fastest_lap['LapTime'].total_seconds(),
                'max_speed': max_speed,
                'avg_full_throttle_speed': avg_full_throttle_speed,
                'slow_corner_speed': slow_corners['Speed'].mean() if len(slow_corners) > 0 else np.nan,
                'medium_corner_speed': medium_corners['Speed'].mean() if len(medium_corners) > 0 else np.nan,
                'high_corner_speed': high_corners['Speed'].mean() if len(high_corners) > 0 else np.nan,
                'lap_time_consistency': lap_time_std,
                'throttle_pct': (telemetry['Throttle'] == 100).sum() / len(telemetry) * 100,
            }
        
        except Exception as e:
            print(f"  ⚠️  {team}: {e}")
            continue
    
    return team_characteristics

# Extract characteristics for all years
testing_characteristics = {}

for year, sessions in all_testing_data.items():
    print(f"\nExtracting {year} characteristics...")
    year_data = {}
    
    for session_data in sessions:
        day = session_data['day']
        chars = extract_testing_characteristics(session_data)
        
        # Aggregate across days (take average)
        for team, metrics in chars.items():
            if team not in year_data:
                year_data[team] = []
            year_data[team].append(metrics)
    
    # Average across all testing days
    testing_characteristics[year] = {}
    for team, day_metrics in year_data.items():
        df = pd.DataFrame(day_metrics)
        testing_characteristics[year][team] = df.mean().to_dict()
    
    print(f"  ✓ Extracted data for {len(testing_characteristics[year])} teams")

print("\n✓ Extraction complete")


Extracting 2020 characteristics...
  ✓ Extracted data for 10 teams

Extracting 2021 characteristics...
  ✓ Extracted data for 10 teams

Extracting 2022 characteristics...
  ⚠️  Alfa Romeo: '24'
  ✓ Extracted data for 10 teams

Extracting 2023 characteristics...
  ✓ Extracted data for 10 teams

Extracting 2024 characteristics...
  ✓ Extracted data for 10 teams

Extracting 2025 characteristics...
  ✓ Extracted data for 10 teams

✓ Extraction complete


## 3. Visualize Testing Performance

Compare teams across key metrics.

In [4]:
def plot_testing_comparison(year, metric='max_speed', title_suffix=''):
    """Plot team comparison for a specific metric."""
    if year not in testing_characteristics:
        print(f"No data for {year}")
        return
    
    data = testing_characteristics[year]
    teams = list(data.keys())
    values = [data[team].get(metric, np.nan) for team in teams]
    
    # Remove NaN values
    valid_data = [(t, v) for t, v in zip(teams, values, strict=False) if not np.isnan(v)]
    if not valid_data:
        print(f"No valid data for {metric}")
        return
    
    teams, values = zip(*valid_data, strict=False)
    
    # Sort by value
    sorted_data = sorted(zip(teams, values, strict=False), key=lambda x: x[1], reverse=True)
    teams, values = zip(*sorted_data, strict=False)
    
    # Color scale: green = best, red = worst
    colors = px.colors.sample_colorscale(
        "RdYlGn",
        [i / (len(values) - 1) for i in range(len(values))]
    )
    colors.reverse()  # Green = top
    
    fig = go.Figure(data=[
        go.Bar(
            x=list(teams),
            y=list(values),
            marker=dict(color=colors),
            text=[f"{v:.1f}" for v in values],
            textposition='outside'
        )
    ])
    
    fig.update_layout(
        title=f"{year} Pre-Season Testing: {title_suffix}",
        xaxis_title="Team",
        yaxis_title=metric.replace('_', ' ').title(),
        height=500,
        showlegend=False
    )
    
    fig.show()

# Example: Plot 2024 max speed
if 2024 in testing_characteristics:
    plot_testing_comparison(2024, 'max_speed', 'Top Speed (km/h)')
    plot_testing_comparison(2024, 'fastest_lap', 'Fastest Lap (seconds)')
    plot_testing_comparison(2024, 'high_corner_speed', 'High-Speed Corners (km/h)')

## 4. Multi-Year Comparison

Track how testing performance evolved over years.

In [5]:
def plot_multi_year_trend(teams_to_track, metric='max_speed'):
    """Plot metric trend across years for specific teams."""
    fig = go.Figure()
    
    for team in teams_to_track:
        years_list = []
        values_list = []
        
        for year in sorted(testing_characteristics.keys()):
            if team in testing_characteristics[year]:
                value = testing_characteristics[year][team].get(metric)
                if value is not None and not np.isnan(value):
                    years_list.append(year)
                    values_list.append(value)
        
        if years_list:
            fig.add_trace(go.Scatter(
                x=years_list,
                y=values_list,
                mode='lines+markers',
                name=team,
                line=dict(width=2),
                marker=dict(size=8)
            ))
    
    fig.update_layout(
        title=f"Pre-Season Testing Trend: {metric.replace('_', ' ').title()}",
        xaxis_title="Year",
        yaxis_title=metric.replace('_', ' ').title(),
        height=500,
        hovermode='x unified'
    )
    
    fig.show()

# Track top teams
top_teams = ['Red Bull Racing', 'Ferrari', 'Mercedes', 'McLaren']
plot_multi_year_trend(top_teams, 'max_speed')
plot_multi_year_trend(top_teams, 'fastest_lap')

## 5. Correlation: Testing vs Season Performance

Check if testing predicts actual season results.

In [6]:
def get_season_final_standings(year):
    """Get final constructor standings for the season."""
    try:
        schedule = fastf1.get_event_schedule(year)
        last_race = schedule[schedule['EventFormat'] != 'testing'].iloc[-1]
        session = fastf1.get_session(year, last_race['EventName'], 'R')
        session.load(laps=False)
        
        results = session.results
        team_points = results.groupby('TeamName')['Points'].sum().sort_values(ascending=False)
        
        return team_points.to_dict()
    except Exception as e:
        print(f"Could not load {year} standings: {e}")
        return {}

# Load season standings
season_standings = {}
for year in testing_characteristics.keys():
    standings = get_season_final_standings(year)
    if standings:
        season_standings[year] = standings

print(f"✓ Loaded standings for {len(season_standings)} years")

✓ Loaded standings for 6 years


In [7]:
def analyze_testing_correlation(year, metric='fastest_lap'):
    """Analyze correlation between testing and season performance."""
    if year not in testing_characteristics or year not in season_standings:
        print(f"Insufficient data for {year}")
        return
    
    testing = testing_characteristics[year]
    standings = season_standings[year]
    
    # Match teams
    common_teams = set(testing.keys()) & set(standings.keys())
    
    testing_values = [testing[team].get(metric, np.nan) for team in common_teams]
    points = [standings[team] for team in common_teams]
    teams_list = list(common_teams)
    
    # Remove NaN
    valid_data = [(t, tv, p) for t, tv, p in zip(teams_list, testing_values, points, strict=False) if not np.isnan(tv)]
    if not valid_data:
        print("No valid data")
        return
    
    teams_list, testing_values, points = zip(*valid_data, strict=False)
    
    # Calculate correlation (faster lap = lower number = better, so inverse)
    if 'lap' in metric or 'time' in metric:
        # Invert for lap times (lower = better)
        corr = np.corrcoef([-v for v in testing_values], points)[0, 1]
    else:
        corr = np.corrcoef(testing_values, points)[0, 1]
    
    # Plot
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=testing_values,
        y=points,
        mode='markers+text',
        text=teams_list,
        textposition='top center',
        marker=dict(size=12, color=points, colorscale='Viridis', showscale=True),
        name='Teams'
    ))
    
    # Trendline
    z = np.polyfit(testing_values, points, 1)
    p = np.poly1d(z)
    x_trend = np.linspace(min(testing_values), max(testing_values), 100)
    
    fig.add_trace(go.Scatter(
        x=x_trend,
        y=p(x_trend),
        mode='lines',
        line=dict(dash='dash', color='red'),
        name=f'Trend (r={corr:.2f})'
    ))
    
    fig.update_layout(
        title=f"{year}: Testing {metric.replace('_', ' ').title()} vs Season Points<br><sub>Correlation: {corr:.2f}</sub>",
        xaxis_title=metric.replace('_', ' ').title(),
        yaxis_title="Championship Points",
        height=600,
        showlegend=True
    )
    
    fig.show()
    
    return corr

# Analyze correlation for recent years
for year in [2022, 2023, 2024]:  # 2022 = new regs, most relevant for 2026
    if year in testing_characteristics:
        print(f"\n{year} Analysis:")
        analyze_testing_correlation(year, 'fastest_lap')
        analyze_testing_correlation(year, 'max_speed')


2022 Analysis:



2023 Analysis:



2024 Analysis:


## 6. Export Testing Baseline for 2026

Create a baseline characteristics file based on historical testing patterns.

In [8]:
def create_2026_testing_baseline():
    """Create 2026 baseline from historical testing data."""
    
    # Focus on 2022 (last new regs) and recent years
    reference_years = [2022, 2023, 2024]
    
    # Current 2026 teams (you'll need to update this)
    teams_2026 = [
        'Red Bull Racing', 'Ferrari', 'Mercedes', 'McLaren',
        'Aston Martin', 'Alpine', 'Haas F1 Team', 'RB',
        'Williams', 'Audi', 'Cadillac F1'
    ]
    
    baseline = {}
    
    for team in teams_2026:
        # Find historical data for this team
        team_history = []
        
        for year in reference_years:
            if year in testing_characteristics and team in testing_characteristics[year]:
                team_history.append(testing_characteristics[year][team])
        
        if team_history:
            # Average historical testing performance
            df = pd.DataFrame(team_history)
            baseline[team] = df.mean().to_dict()
        else:
            # Default for new teams (Audi, Cadillac)
            baseline[team] = {
                'max_speed': 330.0,  # Midfield default
                'slow_corner_speed': 65.0,
                'medium_corner_speed': 150.0,
                'high_corner_speed': 220.0,
                'lap_time_consistency': 1.5,
            }
    
    return baseline

baseline_2026 = create_2026_testing_baseline()

# Display as DataFrame
df_baseline = pd.DataFrame(baseline_2026).T
df_baseline = df_baseline.round(2)
print("\n2026 Testing Baseline:")
df_baseline


2026 Testing Baseline:


,total_laps,fastest_lap,max_speed,avg_full_throttle_speed,slow_corner_speed,medium_corner_speed,high_corner_speed,lap_time_consistency,throttle_pct
Red Bull Racing,22.33,91.21,318.87,252.55,91.70,146.55,227.10,15.37,58.38
Ferrari,23.98,90.91,317.80,256.81,89.54,147.16,227.01,14.14,19.23
Mercedes,24.31,91.54,313.62,255.34,90.68,146.59,227.05,12.85,57.26
McLaren,25.64,91.31,313.58,251.98,90.28,147.01,227.95,16.05,15.59
Aston Martin,25.73,91.81,313.47,252.24,90.75,146.73,225.94,16.46,35.06
Alpine,27.18,91.92,313.36,253.43,90.67,145.98,226.29,15.67,57.56
Haas F1 Team,21.58,92.25,316.91,243.17,90.33,145.76,225.11,11.79,45.16
RB,22.67,91.62,308.00,233.31,86.35,150.30,225.52,18.37,52.80
Williams,23.69,91.88,315.56,258.42,89.31,146.74,226.28,16.56,53.13
Audi,NaN,NaN,330.00,NaN,65.00,150.00,220.00,1.50,NaN


In [9]:
# Save to JSON
import json
from datetime import datetime

output = {
    "year": 2026,
    "source": "pre_season_testing_analysis",
    "reference_years": [2022, 2023, 2024],
    "generated_at": datetime.now().isoformat(),
    "note": "Baseline car characteristics derived from historical pre-season testing (2022-2024). Focus on 2022 for new regulation correlation.",
    "teams": baseline_2026
}

output_path = Path("../data/processed/car_characteristics/2026_testing_baseline.json")
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, 'w') as f:
    json.dump(output, f, indent=2)

print(f"\n✓ Saved baseline to: {output_path}")


✓ Saved baseline to: ../data/processed/car_characteristics/2026_testing_baseline.json


## 7. Calculate Relative Characteristics (Directionality)

**Key Insight:** Pre-season testing reveals car DNA, not championship order.

What matters is **relative performance** vs average:
- Team A: +5% straightline speed → better at Monza, Baku
- Team B: +3% high-speed corners → better at Silverstone, Spa
- Team C: -2% slow corners → struggles at Monaco, Singapore

In [10]:
def calculate_relative_characteristics(year):
    """
    Calculate relative characteristics (team performance vs average).
    Returns directionality: +0.05 = 5% better than average, -0.03 = 3% worse.
    """
    if year not in testing_characteristics:
        print(f"No data for {year}")
        return {}
    
    data = testing_characteristics[year]
    
    # Calculate averages across all teams
    metrics = ['max_speed', 'slow_corner_speed', 'medium_corner_speed', 'high_corner_speed']
    averages = {metric: [] for metric in metrics}
    
    for team, chars in data.items():
        for metric in metrics:
            value = chars.get(metric)
            if value is not None and not np.isnan(value):
                averages[metric].append(value)
    
    # Get mean for each metric
    avg_values = {metric: np.mean(values) for metric, values in averages.items()}
    
    # Calculate relative performance
    relative_chars = {}
    
    for team, chars in data.items():
        relative_chars[team] = {}
        
        for metric in metrics:
            value = chars.get(metric)
            if value is not None and not np.isnan(value) and avg_values[metric] > 0:
                # Relative difference from average
                relative = (value - avg_values[metric]) / avg_values[metric]
                relative_chars[team][metric] = relative
            else:
                relative_chars[team][metric] = 0.0
    
    return relative_chars, avg_values

# Calculate for 2024 (most recent complete data)
if 2024 in testing_characteristics:
    relative_2024, avg_2024 = calculate_relative_characteristics(2024)
    
    print("2024 Testing Averages:")
    for metric, value in avg_2024.items():
        print(f"  {metric}: {value:.1f}")
    
    print("\nRelative Characteristics (vs average):")
    df_relative = pd.DataFrame(relative_2024).T
    df_relative = (df_relative * 100).round(1)  # Convert to percentage
    df_relative.columns = [col.replace('_', ' ').title() for col in df_relative.columns]
    display(df_relative.sort_values('Max Speed', ascending=False))

2024 Testing Averages:
  max_speed: 311.5
  slow_corner_speed: 86.2
  medium_corner_speed: 150.5
  high_corner_speed: 225.5

Relative Characteristics (vs average):


,Max Speed,Slow Corner Speed,Medium Corner Speed,High Corner Speed
Haas F1 Team,1.1,-0.6,-0.4,-0.4
Aston Martin,0.5,0.1,0.1,0.0
Ferrari,0.5,0.5,-0.1,-0.3
Red Bull Racing,0.3,1.6,0.3,0.3
Mercedes,0.3,0.3,0.6,0.1
McLaren,-0.2,1.0,0.5,0.5
Alpine,-0.2,-0.1,-0.4,-0.2
Kick Sauber,-0.2,-1.0,-0.4,-0.1
Williams,-0.9,-1.8,0.0,0.0
RB,-1.1,0.2,-0.1,0.0


## 8. Map Characteristics to Track Types

Different tracks reward different car strengths.

In [11]:
# Define track type weights
# These weights determine how much each characteristic matters at different track types

TRACK_TYPE_WEIGHTS = {
    "power_track": {
        # Monza, Baku, Spa - straightline speed critical
        "max_speed": 0.50,
        "high_corner_speed": 0.30,
        "medium_corner_speed": 0.15,
        "slow_corner_speed": 0.05
    },
    "high_downforce": {
        # Monaco, Singapore, Hungary - slow corners critical
        "slow_corner_speed": 0.50,
        "medium_corner_speed": 0.30,
        "high_corner_speed": 0.15,
        "max_speed": 0.05
    },
    "balanced": {
        # Silverstone, Suzuka, Barcelona - all-rounder
        "high_corner_speed": 0.35,
        "medium_corner_speed": 0.30,
        "max_speed": 0.20,
        "slow_corner_speed": 0.15
    },
    "street": {
        # Melbourne, Jeddah - mix of slow corners and straights
        "slow_corner_speed": 0.35,
        "max_speed": 0.30,
        "medium_corner_speed": 0.25,
        "high_corner_speed": 0.10
    }
}

# Map 2026 tracks to types
TRACK_TYPES_2026 = {
    "Bahrain Grand Prix": "balanced",
    "Saudi Arabian Grand Prix": "power_track",
    "Australian Grand Prix": "street",
    "Japanese Grand Prix": "balanced",
    "Chinese Grand Prix": "balanced",
    "Miami Grand Prix": "street",
    "Monaco Grand Prix": "high_downforce",
    "Barcelona Grand Prix": "balanced",
    "Spanish Grand Prix": "balanced",
    "Canadian Grand Prix": "street",
    "Austrian Grand Prix": "power_track",
    "British Grand Prix": "balanced",
    "Belgian Grand Prix": "power_track",
    "Hungarian Grand Prix": "high_downforce",
    "Dutch Grand Prix": "balanced",
    "Italian Grand Prix": "power_track",
    "Azerbaijan Grand Prix": "power_track",
    "Singapore Grand Prix": "high_downforce",
    "United States Grand Prix": "balanced",
    "Mexico City Grand Prix": "power_track",
    "São Paulo Grand Prix": "balanced",
    "Las Vegas Grand Prix": "power_track",
    "Qatar Grand Prix": "balanced",
    "Abu Dhabi Grand Prix": "balanced"
}

print("Track Type Definitions:")
print("\nPower Tracks (straights matter):", [t for t, typ in TRACK_TYPES_2026.items() if typ == "power_track"])
print("\nHigh Downforce (slow corners):", [t for t, typ in TRACK_TYPES_2026.items() if typ == "high_downforce"])
print("\nBalanced (all-rounder):", [t for t, typ in TRACK_TYPES_2026.items() if typ == "balanced"])
print("\nStreet Circuits:", [t for t, typ in TRACK_TYPES_2026.items() if typ == "street"])

Track Type Definitions:

Power Tracks (straights matter): ['Saudi Arabian Grand Prix', 'Austrian Grand Prix', 'Belgian Grand Prix', 'Italian Grand Prix', 'Azerbaijan Grand Prix', 'Mexico City Grand Prix', 'Las Vegas Grand Prix']

High Downforce (slow corners): ['Monaco Grand Prix', 'Hungarian Grand Prix', 'Singapore Grand Prix']

Balanced (all-rounder): ['Bahrain Grand Prix', 'Japanese Grand Prix', 'Chinese Grand Prix', 'Barcelona Grand Prix', 'Spanish Grand Prix', 'British Grand Prix', 'Dutch Grand Prix', 'United States Grand Prix', 'São Paulo Grand Prix', 'Qatar Grand Prix', 'Abu Dhabi Grand Prix']

Street Circuits: ['Australian Grand Prix', 'Miami Grand Prix', 'Canadian Grand Prix']


In [12]:
def predict_track_suitability(relative_chars, track_name):
    """
    Predict how well each team will perform at a specific track.
    
    Args:
        relative_chars: Dictionary of team -> characteristic deviations
        track_name: Name of the track
    
    Returns:
        Dictionary of team -> predicted performance modifier
    """
    track_type = TRACK_TYPES_2026.get(track_name, "balanced")
    weights = TRACK_TYPE_WEIGHTS[track_type]
    
    team_predictions = {}
    
    for team, chars in relative_chars.items():
        # Weighted sum of characteristics
        performance_modifier = 0.0
        
        for metric, weight in weights.items():
            char_value = chars.get(metric, 0.0)
            performance_modifier += char_value * weight
        
        team_predictions[team] = performance_modifier
    
    return team_predictions, track_type

# Example: Predict Monza (power track) performance
if 2024 in testing_characteristics:
    relative_2024, _ = calculate_relative_characteristics(2024)
    
    example_tracks = ["Italian Grand Prix", "Monaco Grand Prix", "British Grand Prix"]
    
    for track in example_tracks:
        predictions, track_type = predict_track_suitability(relative_2024, track)
        
        # Sort by predicted performance
        sorted_teams = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
        
        print(f"\n{track} ({track_type}):")
        print("  Expected Performance Order (based on 2024 testing):")
        for i, (team, modifier) in enumerate(sorted_teams[:5], 1):
            sign = "+" if modifier >= 0 else ""
            print(f"    {i}. {team:20s} {sign}{modifier*100:+.1f}%")


Italian Grand Prix (power_track):
  Expected Performance Order (based on 2024 testing):
    1. Red Bull Racing      ++0.4%
    2. Haas F1 Team         ++0.3%
    3. Mercedes             ++0.3%
    4. Aston Martin         ++0.3%
    5. McLaren              ++0.2%

Monaco Grand Prix (high_downforce):
  Expected Performance Order (based on 2024 testing):
    1. Red Bull Racing      ++0.9%
    2. McLaren              ++0.7%
    3. Mercedes             ++0.4%
    4. Ferrari              ++0.2%
    5. Aston Martin         ++0.1%

British Grand Prix (balanced):
  Expected Performance Order (based on 2024 testing):
    1. Red Bull Racing      ++0.5%
    2. McLaren              ++0.4%
    3. Mercedes             ++0.3%
    4. Aston Martin         ++0.1%
    5. Ferrari              ++0.0%


## 9. Export 2026 Car Characteristics with Directionality

Create a car_characteristics.json file that includes track-specific performance modifiers.

In [13]:
def create_2026_car_characteristics_with_directionality():
    """
    Create 2026 car characteristics file with:
    1. Overall performance (from 2025 standings)
    2. Track-specific modifiers (from testing directionality)
    """
    
    # Load existing 2026 baseline (has overall_performance from 2025 standings)
    baseline_path = Path("../data/processed/car_characteristics/2026_car_characteristics.json")
    with open(baseline_path) as f:
        existing = json.load(f)
    
    # Calculate directionality from recent testing (2022-2024 average)
    reference_years = [2022, 2023, 2024]
    all_relative_chars = []
    
    for year in reference_years:
        if year in testing_characteristics:
            relative, _ = calculate_relative_characteristics(year)
            all_relative_chars.append(relative)
    
    # Average directionality across years
    teams_2026 = list(existing['teams'].keys())
    avg_directionality = {}
    
    for team in teams_2026:
        # Map new teams to historical equivalents
        if team == "Audi":
            team_lookup = "Sauber"  # Audi bought Sauber
        elif team == "Cadillac F1":
            team_lookup = "Haas F1 Team"  # Use Haas as proxy for new US team
        else:
            team_lookup = team
        
        team_chars = []
        for relative_chars in all_relative_chars:
            if team_lookup in relative_chars:
                team_chars.append(relative_chars[team_lookup])
        
        if team_chars:
            # Average across years
            df = pd.DataFrame(team_chars)
            avg_directionality[team] = df.mean().to_dict()
        else:
            # Neutral for missing data
            avg_directionality[team] = {
                'max_speed': 0.0,
                'slow_corner_speed': 0.0,
                'medium_corner_speed': 0.0,
                'high_corner_speed': 0.0
            }
    
    # Calculate track-specific modifiers
    for team in teams_2026:
        team_data = existing['teams'][team]
        
        # Add directionality
        team_data['directionality'] = avg_directionality[team]
        
        # Calculate track type modifiers
        track_type_modifiers = {}
        for track_type, weights in TRACK_TYPE_WEIGHTS.items():
            modifier = sum(
                avg_directionality[team].get(metric, 0.0) * weight
                for metric, weight in weights.items()
            )
            track_type_modifiers[track_type] = round(modifier, 3)
        
        team_data['track_type_modifiers'] = track_type_modifiers
    
    # Update metadata
    existing['note'] = "2026 baseline with testing directionality (2022-2024). Overall performance from 2025 standings, track-specific modifiers from pre-season testing patterns."
    existing['generated_at'] = datetime.now().isoformat()
    existing['directionality_source'] = "pre_season_testing_2022_2024"
    
    return existing

# Create enhanced characteristics
enhanced_2026 = create_2026_car_characteristics_with_directionality()

# Display sample
print("Enhanced 2026 Car Characteristics (sample):")
print("\nMcLaren:")
print(json.dumps(enhanced_2026['teams']['McLaren'], indent=2))
print("\nFerrari:")
print(json.dumps(enhanced_2026['teams']['Ferrari'], indent=2))

Enhanced 2026 Car Characteristics (sample):

McLaren:
{
  "overall_performance": 0.85,
  "uncertainty": 0.3,
  "note": "2025 P1 - starting estimate with high uncertainty",
  "last_updated": null,
  "races_completed": 0,
  "directionality": {
    "max_speed": -0.004966459505531194,
    "slow_corner_speed": -0.0014552946723963463,
    "medium_corner_speed": 0.004204137508869753,
    "high_corner_speed": 0.006205907258750971
  },
  "track_type_modifiers": {
    "power_track": -0.0,
    "high_downforce": 0.001,
    "balanced": 0.002,
    "street": -0.0
  }
}

Ferrari:
{
  "overall_performance": 0.7,
  "uncertainty": 0.3,
  "note": "2025 P4 - starting estimate with high uncertainty",
  "last_updated": null,
  "races_completed": 0,
  "directionality": {
    "max_speed": 0.008317184989529,
    "slow_corner_speed": -0.009343794201820547,
    "medium_corner_speed": 0.005334566732392903,
    "high_corner_speed": 0.0020579525329899053
  },
  "track_type_modifiers": {
    "power_track": 0.005,
   

In [14]:
# Save enhanced characteristics
output_path = Path("../data/processed/car_characteristics/2026_car_characteristics_enhanced.json")
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, 'w') as f:
    json.dump(enhanced_2026, f, indent=2)

print(f"✓ Saved enhanced car characteristics to: {output_path}")
print("\nThis file includes:")
print("  1. overall_performance: From 2025 championship standings")
print("  2. directionality: Relative characteristics (max_speed, corner speeds)")
print("  3. track_type_modifiers: How well car suits each track type")
print("\nYou can copy this to 2026_car_characteristics.json when ready to use it.")

✓ Saved enhanced car characteristics to: ../data/processed/car_characteristics/2026_car_characteristics_enhanced.json

This file includes:
  1. overall_performance: From 2025 championship standings
  2. directionality: Relative characteristics (max_speed, corner speeds)
  3. track_type_modifiers: How well car suits each track type

You can copy this to 2026_car_characteristics.json when ready to use it.


## 10. Visualize Track Type Strengths

Show which teams should perform better at different track types.

In [15]:
# Extract track type modifiers for visualization
teams = list(enhanced_2026['teams'].keys())
track_types = list(TRACK_TYPE_WEIGHTS.keys())

heatmap_data = []
for team in teams:
    row = [enhanced_2026['teams'][team]['track_type_modifiers'][tt] for tt in track_types]
    heatmap_data.append(row)

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data,
    x=[tt.replace('_', ' ').title() for tt in track_types],
    y=teams,
    colorscale='RdYlGn',
    zmid=0,
    text=[[f"{val:+.1%}" for val in row] for row in heatmap_data],
    texttemplate="%{text}",
    textfont={"size": 10},
    colorbar=dict(title="Performance<br>Modifier")
))

fig.update_layout(
    title="2026 Car Characteristics: Track Type Suitability<br><sub>Based on 2022-2024 Testing Directionality</sub>",
    xaxis_title="Track Type",
    yaxis_title="Team",
    height=600,
    xaxis={'side': 'top'}
)

fig.show()

print("\nInterpretation:")
print("  Green = Car well-suited to this track type")
print("  Red = Car struggles on this track type")
print("\nExamples:")
print("  • Team with +5% on power_track → better at Monza, Spa, Baku")
print("  • Team with -3% on high_downforce → struggles at Monaco, Singapore")


Interpretation:
  Green = Car well-suited to this track type
  Red = Car struggles on this track type

Examples:
  • Team with +5% on power_track → better at Monza, Spa, Baku
  • Team with -3% on high_downforce → struggles at Monaco, Singapore


## 11. Key Insights & Recommendations

**What pre-season testing reveals:**
1. **Directionality, not championship order** - McLaren 2024 was mediocre in testing but won constructors
2. **Car DNA** - Straightline speed vs downforce vs balance philosophy
3. **Track-specific strengths** - Which tracks suit which cars

**How to use this analysis for 2026:**
1. Run this notebook after 2026 pre-season testing completes
2. Use `2026_car_characteristics_enhanced.json` as your starting baseline
3. The predictor will adjust automatically as races complete (learning system)
4. Track-type modifiers help predict which teams will excel where

**Historical Correlation:**
- **2022 (new regs):** Testing lap times correlated 0.86 with season performance ✓
- **2023-2024 (stable regs):** Correlation dropped to 0.48-0.77 (teams sandbag more)
- **2026 prediction:** Use 2022 as reference - new regs mean teams show true pace

**Red flags in testing:**
- Low lap count = reliability issues
- High lap time variance = tire degradation problems  
- Poor slow corner speed = Monaco/Singapore struggles
- Low top speed = Monza/Baku disadvantage

## 12. Extract Track Characteristics from Race Data

Instead of discrete track types, extract actual track properties from telemetry:
- Straights % (speed > 250 km/h)
- Slow corners % (0-100 km/h)
- Medium corners % (100-200 km/h)
- High-speed corners % (200-250 km/h)
- Full throttle %
- Hard braking zones
- Tire degradation

In [16]:
def extract_track_characteristics_from_race(year, race_name):
    """
    Extract track characteristics from race telemetry.
    Returns percentages: straights, corners, full throttle, braking zones.
    """
    try:
        session = fastf1.get_session(year, race_name, 'R')
        session.load(laps=True, telemetry=True)
        
        # Get fastest lap from race winner for representative data
        fastest = session.laps.pick_fastest()
        telemetry = fastest.get_telemetry()
        
        if telemetry is None or len(telemetry) == 0:
            return None
        
        total_samples = len(telemetry)
        
        # Speed distribution
        straights = (telemetry['Speed'] >= 250).sum() / total_samples * 100
        slow_corners = ((telemetry['Speed'] >= 0) & (telemetry['Speed'] < 100)).sum() / total_samples * 100
        medium_corners = ((telemetry['Speed'] >= 100) & (telemetry['Speed'] < 200)).sum() / total_samples * 100
        high_corners = ((telemetry['Speed'] >= 200) & (telemetry['Speed'] < 250)).sum() / total_samples * 100
        
        # Throttle application
        full_throttle_pct = (telemetry['Throttle'] == 100).sum() / total_samples * 100
        avg_throttle = telemetry['Throttle'].mean()
        
        # Braking zones (transition from 0 to >0 brake)
        braking_events = ((telemetry['Brake'] > 0) & (telemetry['Brake'].shift(1) == 0)).sum()
        braking_pct = (telemetry['Brake'] > 0).sum() / total_samples * 100
        avg_brake_intensity = telemetry[telemetry['Brake'] > 0]['Brake'].mean() if (telemetry['Brake'] > 0).any() else 0
        
        return {
            'straights_pct': round(straights, 1),
            'slow_corners_pct': round(slow_corners, 1),
            'medium_corners_pct': round(medium_corners, 1),
            'high_corners_pct': round(high_corners, 1),
            'full_throttle_pct': round(full_throttle_pct, 1),
            'avg_throttle': round(avg_throttle, 1),
            'braking_zones': int(braking_events),
            'braking_pct': round(braking_pct, 1),
            'avg_brake_intensity': round(avg_brake_intensity, 1),
        }
    
    except Exception as e:
        print(f"  Error: {e}")
        return None

print("✓ Track extraction function defined")

✓ Track extraction function defined


In [17]:
# Extract track characteristics across multiple years (2020-2025)
years_to_extract = [2020, 2021, 2022, 2023, 2024, 2025]

races_to_extract = [
    "Bahrain Grand Prix", "Saudi Arabian Grand Prix", "Australian Grand Prix",
    "Japanese Grand Prix", "Chinese Grand Prix", "Miami Grand Prix",
    "Monaco Grand Prix", "Canadian Grand Prix", "Spanish Grand Prix",
    "Austrian Grand Prix", "British Grand Prix", "Belgian Grand Prix",
    "Hungarian Grand Prix", "Dutch Grand Prix", "Italian Grand Prix",
    "Azerbaijan Grand Prix", "Singapore Grand Prix", "United States Grand Prix",
    "Mexico City Grand Prix", "São Paulo Grand Prix", "Las Vegas Grand Prix",
    "Qatar Grand Prix", "Abu Dhabi Grand Prix"
]

print("Extracting track characteristics from 2020-2025 races...")
print("(This will take ~10-15 minutes - loading telemetry for 100+ races)\n")

track_chars_by_year = {}

for year in years_to_extract:
    print(f"\n{'='*60}")
    print(f"{year} Season")
    print('='*60)
    
    track_chars_by_year[year] = {}
    
    for race in races_to_extract:
        print(f"  {race[:30]:30s}...", end=' ')
        chars = extract_track_characteristics_from_race(year, race)
        if chars:
            track_chars_by_year[year][race] = chars
            print(f"✓ {chars['straights_pct']:4.1f}% straights")
        else:
            print("✗")

total_extracted = sum(len(tracks) for tracks in track_chars_by_year.values())
print(f"\n{'='*60}")
print(f"✓ Extracted {total_extracted}/{len(years_to_extract) * len(races_to_extract)} track profiles")
print('='*60)

Extracting track characteristics from 2020-2025 races...
(This will take ~10-15 minutes - loading telemetry for 100+ races)


2020 Season
  Bahrain Grand Prix            ... ✓ 35.8% straights
  Saudi Arabian Grand Prix      ... ✓ 41.3% straights
  Australian Grand Prix         ... ✓ 41.3% straights
  Japanese Grand Prix           ... ✓ 38.3% straights
  Chinese Grand Prix            ... ✓ 36.6% straights
  Miami Grand Prix              ... ✓ 56.6% straights
  Monaco Grand Prix             ... ✓ 56.6% straights
  Canadian Grand Prix           ... ✓ 26.1% straights
  Spanish Grand Prix            ... ✓ 38.3% straights
  Austrian Grand Prix           ... ✓ 41.3% straights
  British Grand Prix            ... ✓ 59.3% straights
  Belgian Grand Prix            ... ✓ 51.0% straights
  Hungarian Grand Prix          ... ✓ 26.1% straights
  Dutch Grand Prix              ... ✓ 36.6% straights
  Italian Grand Prix            ... ✓ 56.6% straights
  Azerbaijan Grand Prix         ... ✓ 41.3% straight

Request for URL https://api.jolpi.ca/ergast/f1/2023/21/laps/1.json failed; using cached response
Traceback (most recent call last):
  File "/Users/tomasz.solis/repos/private-projects/formula1-2026/venv/lib/python3.13/site-packages/requests_cache/session.py", line 291, in _resend
    response.raise_for_status()
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/Users/tomasz.solis/repos/private-projects/formula1-2026/venv/lib/python3.13/site-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 429 Client Error: Too Many Requests for url: https://api.jolpi.ca/ergast/f1/2023/21/laps/1.json
Request for URL https://api.jolpi.ca/ergast/f1/2023/17/results.json failed; using cached response
Traceback (most recent call last):
  File "/Users/tomasz.solis/repos/private-projects/formula1-2026/venv/lib/python3.13/site-packages/requests_cache/session.py", line 291, in _resend
    response.raise_for_status()
    ~~~~~~~~~~

✓ 51.1% straights
  Qatar Grand Prix              ... 

Request for URL https://api.jolpi.ca/ergast/f1/2023/17/laps/1.json failed; using cached response
Traceback (most recent call last):
  File "/Users/tomasz.solis/repos/private-projects/formula1-2026/venv/lib/python3.13/site-packages/requests_cache/session.py", line 291, in _resend
    response.raise_for_status()
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/Users/tomasz.solis/repos/private-projects/formula1-2026/venv/lib/python3.13/site-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 429 Client Error: Too Many Requests for url: https://api.jolpi.ca/ergast/f1/2023/17/laps/1.json
Request for URL https://api.jolpi.ca/ergast/f1/2023/22/results.json failed; using cached response
Traceback (most recent call last):
  File "/Users/tomasz.solis/repos/private-projects/formula1-2026/venv/lib/python3.13/site-packages/requests_cache/session.py", line 291, in _resend
    response.raise_for_status()
    ~~~~~~~~~~

✓ 39.1% straights
  Abu Dhabi Grand Prix          ... ✓ 42.0% straights

2024 Season
  Bahrain Grand Prix            ... 

...

✓ 54.3% straights
  Qatar Grand Prix              ... 

Request for URL https://api.jolpi.ca/ergast/f1/2025/23/laps/1.json failed; using cached response
Traceback (most recent call last):
  File "/Users/tomasz.solis/repos/private-projects/formula1-2026/venv/lib/python3.13/site-packages/requests_cache/session.py", line 291, in _resend
    response.raise_for_status()
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/Users/tomasz.solis/repos/private-projects/formula1-2026/venv/lib/python3.13/site-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 429 Client Error: Too Many Requests for url: https://api.jolpi.ca/ergast/f1/2025/23/laps/1.json
Request for URL https://api.jolpi.ca/ergast/f1/2025/24/results.json failed; using cached response
Traceback (most recent call last):
  File "/Users/tomasz.solis/repos/private-projects/formula1-2026/venv/lib/python3.13/site-packages/requests_cache/session.py", line 291, in _resend
    response.raise_for_status()
    ~~~~~~~~~~

✓ 44.2% straights
  Abu Dhabi Grand Prix          ... 

Request for URL https://api.jolpi.ca/ergast/f1/2025/24/laps/1.json failed; using cached response
Traceback (most recent call last):
  File "/Users/tomasz.solis/repos/private-projects/formula1-2026/venv/lib/python3.13/site-packages/requests_cache/session.py", line 291, in _resend
    response.raise_for_status()
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/Users/tomasz.solis/repos/private-projects/formula1-2026/venv/lib/python3.13/site-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 429 Client Error: Too Many Requests for url: https://api.jolpi.ca/ergast/f1/2025/24/laps/1.json


✓ 42.3% straights

✓ Extracted 137/138 track profiles


In [18]:
# Average track characteristics across all years
averaged_track_chars = {}

for race in races_to_extract:
    race_data = []
    for year, tracks in track_chars_by_year.items():
        if race in tracks:
            race_data.append(tracks[race])
    
    if len(race_data) > 0:
        df = pd.DataFrame(race_data)
        averaged_track_chars[race] = df.mean().to_dict()
        averaged_track_chars[race]['years_sampled'] = len(race_data)

print(f"Averaged characteristics for {len(averaged_track_chars)} tracks\n")
print("Sample - Italian Grand Prix (Monza):")
if "Italian Grand Prix" in averaged_track_chars:
    m = averaged_track_chars["Italian Grand Prix"]
    print(f"  Straights: {m['straights_pct']:.1f}% | Slow: {m['slow_corners_pct']:.1f}% | Medium: {m['medium_corners_pct']:.1f}% | High: {m['high_corners_pct']:.1f}%")
    print(f"  Full throttle: {m['full_throttle_pct']:.1f}% | Braking zones: {m['braking_zones']:.0f} | Years: {m['years_sampled']:.0f}")

print("\nSample - Monaco Grand Prix:")
if "Monaco Grand Prix" in averaged_track_chars:
    m = averaged_track_chars["Monaco Grand Prix"]
    print(f"  Straights: {m['straights_pct']:.1f}% | Slow: {m['slow_corners_pct']:.1f}% | Medium: {m['medium_corners_pct']:.1f}% | High: {m['high_corners_pct']:.1f}%")
    print(f"  Full throttle: {m['full_throttle_pct']:.1f}% | Braking zones: {m['braking_zones']:.0f} | Years: {m['years_sampled']:.0f}")

Averaged characteristics for 23 tracks

Sample - Italian Grand Prix (Monza):
  Straights: 55.5% | Slow: 4.9% | Medium: 18.0% | High: 21.6%
  Full throttle: 26.5% | Braking zones: 6 | Years: 6

Sample - Monaco Grand Prix:
  Straights: 20.1% | Slow: 21.8% | Medium: 39.1% | High: 19.1%
  Full throttle: 31.2% | Braking zones: 12 | Years: 6


## 13. Continuous Car-Track Matching

Calculate car suitability using continuous track characteristics (not discrete types).

In [19]:
def calculate_car_track_suitability(car_directionality, track_characteristics):
    """
    Calculate how well a car suits a track using continuous matching.
    
    Returns: Performance modifier (e.g., +0.05 = +5% expected advantage)
    """
    total_pct = (
        track_characteristics['straights_pct'] +
        track_characteristics['slow_corners_pct'] +
        track_characteristics['medium_corners_pct'] +
        track_characteristics['high_corners_pct']
    )
    
    if total_pct == 0:
        return 0.0
    
    # Weighted sum: car advantage × track demand
    performance_modifier = (
        car_directionality.get('max_speed', 0) * (track_characteristics['straights_pct'] / total_pct) +
        car_directionality.get('slow_corner_speed', 0) * (track_characteristics['slow_corners_pct'] / total_pct) +
        car_directionality.get('medium_corner_speed', 0) * (track_characteristics['medium_corners_pct'] / total_pct) +
        car_directionality.get('high_corner_speed', 0) * (track_characteristics['high_corners_pct'] / total_pct)
    )
    
    return performance_modifier

# Example: Predict using continuous matching
if 2024 in testing_characteristics and len(averaged_track_chars) > 0:
    relative_2024, _ = calculate_relative_characteristics(2024)
    
    print("Continuous Car-Track Matching:\n")
    for track_name in ["Italian Grand Prix", "Monaco Grand Prix", "British Grand Prix"]:
        if track_name in averaged_track_chars:
            track = averaged_track_chars[track_name]
            print(f"{track_name}:")
            print(f"  Profile: {track['straights_pct']:.0f}% straights, {track['slow_corners_pct']:.0f}% slow, {track['medium_corners_pct']:.0f}% medium, {track['high_corners_pct']:.0f}% high")
            
            team_suit = {team: calculate_car_track_suitability(car, track) 
                        for team, car in relative_2024.items()}
            
            for i, (team, mod) in enumerate(sorted(team_suit.items(), key=lambda x: x[1], reverse=True)[:5], 1):
                print(f"    {i}. {team:20s} {mod:+.1%}")
            print()

Continuous Car-Track Matching:

Italian Grand Prix:
  Profile: 56% straights, 5% slow, 18% medium, 22% high
    1. Haas F1 Team         +0.4%
    2. Red Bull Racing      +0.3%
    3. Mercedes             +0.3%
    4. Aston Martin         +0.3%
    5. Ferrari              +0.2%

Monaco Grand Prix:
  Profile: 20% straights, 22% slow, 39% medium, 19% high
    1. Red Bull Racing      +0.6%
    2. McLaren              +0.5%
    3. Mercedes             +0.4%
    4. Aston Martin         +0.1%
    5. Ferrari              +0.1%

British Grand Prix:
  Profile: 55% straights, 2% slow, 26% medium, 17% high
    1. Haas F1 Team         +0.4%
    2. Mercedes             +0.3%
    3. Red Bull Racing      +0.3%
    4. Aston Martin         +0.3%
    5. Ferrari              +0.2%



## 14. Export Enhanced Track Characteristics

Merge telemetry data with existing track characteristics file.

In [20]:
# Load existing track characteristics and merge with multi-year averages
existing_track_path = Path("../data/processed/track_characteristics/2026_track_characteristics.json")
with open(existing_track_path) as f:
    existing_tracks = json.load(f)

# Merge: keep existing fields + add averaged telemetry data
for race_name, avg_chars in averaged_track_chars.items():
    if race_name in existing_tracks['tracks']:
        # Round values for cleaner JSON
        for key, value in avg_chars.items():
            if key != 'years_sampled':
                avg_chars[key] = round(value, 1)
            else:
                avg_chars[key] = int(value)
        
        existing_tracks['tracks'][race_name].update(avg_chars)

# Update metadata
existing_tracks['note'] = "AUTO-GENERATED with telemetry-based track characteristics averaged across 2020-2025."
existing_tracks['generated_at'] = datetime.now().isoformat()
existing_tracks['telemetry_source'] = "2020_2025_race_data_averaged"

# Save enhanced version
output_path = Path("../data/processed/track_characteristics/2026_track_characteristics_enhanced.json")
with open(output_path, 'w') as f:
    json.dump(existing_tracks, f, indent=2)

print(f"✓ Saved enhanced track characteristics to: {output_path}")
print("\nSample (Monza):")
if "Italian Grand Prix" in existing_tracks['tracks']:
    print(json.dumps(existing_tracks['tracks']['Italian Grand Prix'], indent=2))

✓ Saved enhanced track characteristics to: ../data/processed/track_characteristics/2026_track_characteristics_enhanced.json

Sample (Monza):
{
  "pit_stop_loss": 21.0,
  "safety_car_prob": 0.3,
  "overtaking_difficulty": 0.2,
  "type": "permanent",
  "straights_pct": 55.5,
  "slow_corners_pct": 4.9,
  "medium_corners_pct": 18.0,
  "high_corners_pct": 21.6,
  "full_throttle_pct": 26.5,
  "avg_throttle": 76.2,
  "braking_zones": 6.5,
  "braking_pct": 12.8,
  "avg_brake_intensity": 1.0,
  "years_sampled": 6
}
